In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-movies-and-tv-shows/netflix_titles.csv


In [3]:
import pandas as pd

# Load the dataset
file_path = "/kaggle/input/netflix-movies-and-tv-shows/netflix_titles.csv"
df = pd.read_csv(file_path)

# Display basic information about the dataset
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


(None,
   show_id     type                  title         director  \
 0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
 1      s2  TV Show          Blood & Water              NaN   
 2      s3  TV Show              Ganglands  Julien Leclercq   
 3      s4  TV Show  Jailbirds New Orleans              NaN   
 4      s5  TV Show           Kota Factory              NaN   
 
                                                 cast        country  \
 0                                                NaN  United States   
 1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
 2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
 3                                                NaN            NaN   
 4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   
 
            date_added  release_year rating   duration  \
 0  September 25, 2021          2020  PG-13     90 min   
 1  September 24, 2021          2021  TV-MA  2 Seasons   
 2  S

In [4]:
# Handling Missing Values

# Fill missing values in 'director' and 'cast' with 'Unknown'
df['director'].fillna('Unknown', inplace=True)
df['cast'].fillna('Unknown', inplace=True)

# Fill missing 'country' with 'Unknown'
df['country'].fillna('Unknown', inplace=True)

# Fill missing 'date_added' with 'Unknown'
df['date_added'].fillna('Unknown', inplace=True)

# Fill missing 'rating' with the mode (most frequent value)
df['rating'].fillna(df['rating'].mode()[0], inplace=True)

# Dropping rows with missing 'duration' as it's crucial for analysis
df.dropna(subset=['duration'], inplace=True)

# Verify missing values are handled
df.isnull().sum()


<ipython-input-4-503e248cfb09>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['director'].fillna('Unknown', inplace=True)
<ipython-input-4-503e248cfb09>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Label Encoding for 'type' (since it's ordinal: Movie/TV Show)
label_encoder = LabelEncoder()
df['type'] = label_encoder.fit_transform(df['type'])

# One-Hot Encoding for 'rating'
df = pd.get_dummies(df, columns=['rating'], prefix='rating')

# One-Hot Encoding for 'listed_in' (splitting multiple genres first)
df['listed_in'] = df['listed_in'].str.split(', ')
genres = df.explode('listed_in')  # Expanding multiple genres into separate rows
df_genres = pd.get_dummies(genres['listed_in'], prefix='genre')  # One-hot encoding genres
df = df.join(df_genres.groupby(df_genres.index).sum())  # Merging genre encoding

# Dropping original 'listed_in' column after encoding
df.drop(columns=['listed_in'], inplace=True)

# Display first few rows after encoding
df.head()


,show_id,type,title,director,cast,country,date_added,release_year,duration,description,...,genre_TV Action & Adventure,genre_TV Comedies,genre_TV Dramas,genre_TV Horror,genre_TV Mysteries,genre_TV Sci-Fi & Fantasy,genre_TV Shows,genre_TV Thrillers,genre_Teen TV Shows,genre_Thrillers
0,s1,0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,90 min,"As her father nears the end of his life, filmm...",...,0,0,0,0,0,0,0,0,0,0
1,s2,1,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,2 Seasons,"After crossing paths at a party, a Cape Town t...",...,0,0,1,0,1,0,0,0,0,0
2,s3,1,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,"September 24, 2021",2021,1 Season,To protect his family from a powerful drug lor...,...,1,0,0,0,0,0,0,0,0,0
3,s4,1,Jailbirds New Orleans,Unknown,Unknown,Unknown,"September 24, 2021",2021,1 Season,"Feuds, flirtations and toilet talk go down amo...",...,0,0,0,0,0,0,0,0,0,0
4,s5,1,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,2 Seasons,In a city of coaching centers known to train I...,...,0,1,0,0,0,0,0,0,0,0


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Convert 'duration' to numerical values
def convert_duration(value):
    if 'min' in value:
        return int(value.replace(' min', ''))
    elif 'Season' in value:
        return int(value.split(' ')[0]) * 60  # Approximate 1 season as 60 minutes
    return 0

df['duration'] = df['duration'].apply(convert_duration)

# Min-Max Scaling for 'release_year' and 'duration'
scaler = MinMaxScaler()
df[['release_year', 'duration']] = scaler.fit_transform(df[['release_year', 'duration']])

# Display first few rows after scaling
df[['release_year', 'duration']].head()


,release_year,duration
0,0.989583,0.085546
1,1.000000,0.115044
2,1.000000,0.056047
3,1.000000,0.056047
4,1.000000,0.115044


In [7]:
from sklearn.model_selection import train_test_split

# Selecting relevant features
X = df.drop(columns=['show_id', 'title', 'director', 'cast', 'country', 'date_added', 'description'])
y = df['type']  # Target variable (Movie or TV Show)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display dataset shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape



((7043, 59), (1761, 59), (7043,), (1761,))